In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('resources/train.csv')

In [3]:
df.head()

,user_id,restaurant_id,type,timestamp
0,2,17782,check-in,17/10/2012 11:03
1,6219,22458,check-in,26/10/2012 14:34
2,3,54559,check-in,28/10/2012 13:06
3,4,3377,check-in,29/10/2012 20:21
4,4,27473,check-in,29/10/2012 20:56


In [12]:
temp = df.groupby(['restaurant_id']).size()
drop_index = temp.loc[temp.values > 50].index.array
df = df.loc[df['restaurant_id'].isin(drop_index)]

In [13]:
df['score'] = df.apply(lambda row: 2 if row['type']=='check-in' else 1 , axis = 1)

In [14]:
train_data = df.drop(columns=['type','timestamp'])

In [15]:
def calculate_score(df):
    return df['score'].sum()

In [16]:
temp = train_data.groupby(['restaurant_id','user_id']).apply(calculate_score)

In [17]:
temp = pd.DataFrame(temp).reset_index()

In [18]:
temp.columns = ['restaurant_id', 'user_id', 'score']

In [19]:
temp.to_csv(path_or_buf='resources/filtered_train.csv', index=False)

In [20]:
temp['score'] = 1

In [22]:
temp.to_csv(path_or_buf='resources/binary_filtered_train.csv', index=False)

## EX

In [2]:
train = pd.read_csv('resources/train.csv')

In [3]:
temp = train.drop(['type','timestamp'], axis=1).drop_duplicates()

In [4]:
small_user = temp.groupby(['user_id']).size().reset_index()
small_user.columns = ['user_id','num_restaurant']
small_user_l = small_user[small_user['num_restaurant']==1]['user_id'].to_numpy()

In [5]:
small_restaurant = temp.groupby(['restaurant_id']).size().reset_index()
small_restaurant.columns = ['restaurant_id', 'num_user']
small_restaurant_l = small_restaurant[small_restaurant['num_user']<50]['restaurant_id'].to_numpy()

In [6]:
filtered_train = train[~train['user_id'].isin(small_user_l)]
filtered_train = filtered_train[~filtered_train['restaurant_id'].isin(small_restaurant_l)]

In [7]:
filtered_train = filtered_train.drop(['timestamp'], axis=1)

In [8]:
filtered_train['score'] = filtered_train.apply(lambda row: 2 if row['type']=='check-in' else 1 , axis = 1)

In [9]:
filtered_train = filtered_train.drop(['type'], axis=1)

In [10]:
filtered_train = filtered_train.groupby(['restaurant_id', 'user_id'])['score'].agg('sum').reset_index()

In [11]:
filtered_train.to_csv(path_or_buf='resources/filtered_train_ex.csv', index=False)